In [ ]:
!pip install -U langchain langchain-community faiss-cpu sentence-transformers transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 137.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorDB = FAISS.load_local("/content/drive/MyDrive/VDb", embedding_model, allow_dangerous_deserialization=True)
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

In [ ]:
# Query input
query = "Widening to two lane with paved shoulder of Raipur- Jassakhera"

# Step 1: Retrieve top K documents
docs = vectorDB.similarity_search(query, k=10)

# Step 2: Rerank
pairs = [(query, doc.page_content) for doc in docs]
scores = reranker.predict(pairs)

# Step 3: Sort and get top 3 docs
top_docs = [doc for _, doc in sorted(zip(scores, docs), key=lambda x: x[0], reverse=True)][:3]

# Step 4: Print each document chunk with its source
print("✅ Top 3 Reranked Chunks with Sources:\n")
for i, doc in enumerate(top_docs, 1):
    source = doc.metadata.get("source", "Unknown source")
    print(f"📄 Source {i}: {source}")
    print(doc.page_content[:1000])  # Truncate to 1000 characters if needed
    print("\n" + "-" * 80 + "\n")

# Step 5: Combine chunks for full context
combined_context = "\n\n".join(doc.page_content[:1000] for doc in top_docs)




✅ Top 3 Reranked Chunks with Sources:

📄 Source 1: 7. DCF bhilwara letter.pdf
--12024//3/
fic23-05-202y
3y
faqy:-
Widening to two lane with paved shoulder of Raipur-jassakhera from
km0.000to33.900
NH-458
(proposal
no.
FP/RAJ/ROAD/451083/2023)
yRi:-
319523.18.9.2024
39 fi y  
.33
..3
y133522.9.20153195692
   f    f
3.763,774875912
69.363y3
FCA Gen.

--------------------------------------------------------------------------------

📄 Source 2: brief note 458.pdf
(Status as on 08.10.2024) 
 
BRIEF-NOTE 
 
1. Proposal name Widening to two lane with paved shoulder of Raipur- Jassakhera from km 0.000 to 
33.900 of NH-458. 
2 Forest Clearance Proposal No. FP/RJ/ROAD/451083/2023) 
3 Forest area to be diverted:  23.52 Ha. 
4 Existing Road Length 33.900 Kms ( 32.500 Kms in Beawar Distt & 1.500 Kms in Rajasamand District. ) 
5 Proposed Project Length 30.31 Kms 
6 Status of DPR  Submitted to MoRTH Jaipur office. 
Project brief :- 
  The above work has been considered in annual plan 2024-25 of Minis

In [ ]:
print("🧠 Combined Context for LLM:\n")
print(combined_context)

🧠 Combined Context for LLM:

--12024//3/
fic23-05-202y
3y
faqy:-
Widening to two lane with paved shoulder of Raipur-jassakhera from
km0.000to33.900
NH-458
(proposal
no.
FP/RAJ/ROAD/451083/2023)
yRi:-
319523.18.9.2024
39 fi y  
.33
..3
y133522.9.20153195692
   f    f
3.763,774875912
69.363y3
FCA Gen.

(Status as on 08.10.2024) 
 
BRIEF-NOTE 
 
1. Proposal name Widening to two lane with paved shoulder of Raipur- Jassakhera from km 0.000 to 
33.900 of NH-458. 
2 Forest Clearance Proposal No. FP/RJ/ROAD/451083/2023) 
3 Forest area to be diverted:  23.52 Ha. 
4 Existing Road Length 33.900 Kms ( 32.500 Kms in Beawar Distt & 1.500 Kms in Rajasamand District. ) 
5 Proposed Project Length 30.31 Kms 
6 Status of DPR  Submitted to MoRTH Jaipur office. 
Project brief :- 
  The above work has been considered in annual plan 2024-25 of Ministry of Road Transportation & Highway, Delhi for 
construction purpose. Accordingly detailed project report has been prepared in parts. First part is having length